In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_excel('../dataframe2.xlsx', sheet_name='Sheet1')
df

In [ ]:
df.drop(['Unnamed: 0','percentage of hard mode','%','easywords','hardwords','easywordsband','hardwordsband'], axis=1, inplace=True)
df = df.sort_values('Contest number', ascending=True)
df.drop('Contest number', axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
data = df.iloc[:, 1:10].copy().reset_index()
data

In [ ]:
data = data.iloc[:, 1:2].copy()
data_origin = data.copy()
data

In [ ]:
plt.figure(figsize=(5, 3))
data[:].plot()
plt.show()

In [ ]:
# 取差作为输入
def difference(data_set,interval=1):
    diff=list()
    for i in range(interval,len(data_set)):
        value=data_set[i]-data_set[i-interval]
        diff.append(value)
    return pd.Series(diff)
 
# 这里的series是之前数据预处理后得到的DateFrame型数据
data=data.values
data_diff=difference(data,1)
data_diff

In [ ]:
def timeseries_to_supervised(data,lag=1):
    df=pd.DataFrame(data)
    columns=[df.shift(1)]
    columns.append(df)
    df=pd.concat(columns,axis=1)
    df.fillna(0,inplace=True)
    return df
 
seq=timeseries_to_supervised(data_diff, 1)
data_seq=seq.values
data_seq

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
data_seq = scaler.fit_transform(data_seq)

In [ ]:
data_size = len(data_diff)

test_ratio = 0.2
train_size = int(len(data_seq) * (1 - test_ratio))
test_size = len(data_seq) - train_size
data_train, data_test = data_seq[0:train_size], data_seq[train_size:len(data_seq)]

data_train.shape

In [ ]:
X,y=data_train[:,0:-1],data_train[:,-1]
X=X.reshape(X.shape[0],1,X.shape[1])
X.shape, y.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU

 
model=Sequential()
model.add(GRU(4,batch_input_shape=(1,X.shape[1],X.shape[2]),stateful=True))
model.add(Dense(1))

model.compile(loss='mean_squared_error',optimizer='adam', metrics=['mse'])
 
# 构建一个LSTM模型并训练，样本数为1，训练次数为3，LSTM层神经元个数为4
model.fit(X,y,batch_size=1,verbose=1,epochs=5,shuffle=False)

In [ ]:
# 逆缩放
def invert_scale(scaler,X,y):
    new_row=[x for x in X]+[y]
    array=np.array(new_row)
    array=array.reshape(1,len(array))
    invert=scaler.inverse_transform(array)
    return invert[0,-1]

# 逆差分
def invert_difference(history,yhat,interval=1):
    return yhat+history[-interval]

In [ ]:
predictions=list()
for i in range(len(data_test)):

    X,y=data_test[i,0:-1],data_test[i,-1]
    X=X.reshape(1,1,1)

    y_pre = model.predict(X)
    
    y_pre=invert_scale(scaler,X,y_pre)
    y_pre=invert_difference(data,y_pre,len(data_test)+1-i)
    predictions.append(y_pre)

In [ ]:
diff=data[:][-1] - data[:][-2]
true_predict = [data[:][-1]]
for i in range(90):
    model_input = np.zeros((1,2))
    model_input[0,0] = diff
    model_input = scaler.transform(model_input)
    model_input = model_input.reshape(1,1,1,2)[:, :, :, 0]
    y_pre = model.predict(model_input)
    y_pre=invert_scale(scaler,X,y_pre)
    y_pre = y_pre + true_predict[-1]
    true_predict.append(y_pre)
    diff = true_predict[-1] - true_predict[-2]

In [ ]:

# predictions_new = data[:train_size].tolist()+predictions

# true_predict_new = data[:data_size].tolist()+true_predict
# # true_predict = data+true_predict

# plt.plot(true_predict_new)
# plt.plot(data[:])
# plt.plot(predictions_new)
# plt.legend(['predict','data','test'])
# plt.axvline(x = 289,color = 'black',ls = '--')
# plt.axvline(x = 359,color = 'red',ls = ':')
# plt.show()
# plt.savefig('predict.png')


predictions_new = data[:train_size].tolist()+predictions

true_predict_new = data[:data_size].tolist()+true_predict
# true_predict = data+true_predict


plt.plot(true_predict_new)
plt.plot(data[:])
plt.plot(predictions_new)
plt.legend(['predict','data','test'])
plt.axvline(x = 289,color = 'black',ls = '--')
plt.axvline(x = 359,color = 'red',ls = ':')
plt.show()
plt.savefig('predict.png')

In [ ]:
amount = true_predict[-1]
amount